In [ ]:
#处理redis初始化从数据库中加载车辆信息以及定时将redis最新数据同步到数据库
import time
import logging
import configparser
import pika
import socketserver
import socket
from vsail_data_parser import VsailDataParser

In [ ]:
logging.basicConfig(filename="socket-server.log", filemode="w", format="%(asctime)s %(name)s:%(levelname)s:%(message)s", datefmt="%Y-%M-%d %H:%M:%S", level=logging.INFO)
parser = configparser.ConfigParser()
parser.read('application.ini')
logging.info('加载application.ini配置')

#获取rabbitmq配置信息
rq_host = parser.get('rabbitmq', 'host')
#实时消息使用队列名称
rq_real_ex_name = parser.get('rabbitmq', 'rq_real_ex_name')
#历史消息使用队列名称
rq_his_ex_name =  parser.get('rabbitmq', 'rq_his_ex_name')

#获取socket配置信息
sk_host = parser.get('socket', 'host')
sk_port = int(parser.get('socket', 'port'))

In [ ]:
#初始化rabbitmq连接
def init_rabbitmq(host):
    try:
        parameters = pika.ConnectionParameters(rq_host, credentials=pika.credentials.PlainCredentials('admin','admin'), heartbeat=0)
        connection = pika.BlockingConnection(parameters)
        logging.info('rabbitmq已连接')
        return connection
    except Exception as ex:
        logging.exception('rabbitmq初始化错误')
        #异常出错不再往下执行
        raise Exception('rabbitmq初始化错误')

In [ ]:
rqconn = init_rabbitmq(rq_host)
channel = rqconn.channel()

In [ ]:
#创建socket服务
sokt = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sokt.bind((sk_host, sk_port))
sokt.listen(1)
open = True

In [ ]:
while open:
    conn,addr = sokt.accept()
    logging.info("connected by", addr)
    #首次需要校验身份
    flag = True
    #重新尝试次数
    repeat = 0
    while True:
        try:
            cmd = conn.recv(1024)
            message = cmd.decode('utf-8',"ignore")
            if(flag):
                logging.info('校验登陆信息')
                login_user = message.split('#')
                if repeat > 3:
                    logging.info('失败次数太多，请重新连接')
                    conn.sendall('失败次数太多，请重新连接'.encode('utf-8'))
                    conn.close()
                    break
                if len(login_user) != 2:
                    logging.info('非法用户名密码')
                    conn.sendall('非法用户名密码'.encode('utf-8'))
                    repeat += 1;
                    #break
                elif login_user[0] != 'admin':
                    logging.info('用户名不正确')
                    conn.sendall('用户名不正确'.encode('utf-8'))
                    repeat += 1;
                    #break
                elif login_user[1] != 'admin':
                    logging.info('密码不正确')
                    conn.sendall('密码不正确'.encode('utf-8'))
                    repeat += 1;
                    #break
                else:
                    flag = False
                    logging.info('登陆成功，开始接收消息')
                    conn.sendall('OK'.encode('utf-8'))
            else:
                #cmd_status, cmd_res = subprocess.getstatusoutput(cmd.decode('utf-8'))
                
                #判断是否是命令还是应答，如果是应答接收验证成功后需要回复 TODO
                parser = VsailDataParser(message)
                bus_data = parser.translate_to_json()
                #print(bus_data)
                channel.basic_publish(exchange=rq_real_ex_name, routing_key='', body=str(bus_data))
                #channel.basic_publish(exchange=rq_real_ex_name, routing_key='', body=message)
                conn.sendall('OK'.encode('utf-8'))
        except Exception as ex:
            logging.exception('接受消息出错')
            break

In [ ]:
def close():
    global open
    open = False
    sokt.shutdown(2)
    sokt.close()

In [ ]:
close()